# Mathematical Optimisation - Project
This project aims at ricreating the results reported in:

[A fix-and-optimize heuristic for the Unrelated Parallel Machine Scheduling Problem
](https://www.sciencedirect.com/science/article/pii/S0305054823003684)

In [1]:
from Solver import Solver
from MPA.MPA import MPA
from Heuristic.Greedy import Greedy
from Heuristic.FixOpt import FixOpt
import random as r
import json
from IPython.display import clear_output

r.seed(14)

options = {}

# Uncomment if you are using a WLS Gurobi license
# and write your credentials in credentials.txt
with open('credentials.txt') as f: 
    data = f.read() 
options = json.loads(data)

## Creation of the matrices
- P stores the execution times
- S stores the setup times
The matrices are implemented using dictionaries to handle the indexes not starting from 0

### Small instance presented in the paper

In [2]:
P_dict = {
            (1,1): 2,(1,2): 2,(1,3): 3,(1,4): 4,(2,1): 3,(2,2): 3,(2,3): 5,(2,4): 3,
         }

S_dict = {
            (1,1,1): 0,(1,1,2): 2,(1,1,3): 1,(1,1,4): 1,(1,2,1): 2,(1,2,2): 0,(1,2,3): 1,(1,2,4): 2,
            (1,3,1): 3,(1,3,2): 1,(1,3,3): 0,(1,3,4): 3,(1,4,1): 2,(1,4,2): 1,(1,4,3): 2,(1,4,4): 0,
            (2,1,1): 0,(2,1,2): 3,(2,1,3): 2,(2,1,4): 1,(2,2,1): 2,(2,2,2): 0,(2,2,3): 3,(2,2,4): 2,
            (2,3,1): 3,(2,3,2): 2,(2,3,3): 0,(2,3,4): 3,(2,4,1): 2,(2,4,2): 2,(2,4,3): 3,(2,4,4): 0,
         }

### Instance generator
The parameters are:
- The cardinality of the set of jobs N
- The cardinality of the set of machines M
- The minimum and the maximum execution time
- The minimum and the maximum setup time

In [3]:
def generate_instance(N_cardinality, M_cardinality,
                      min_execution_time = 1,max_execution_time = 100,
                      min_setup_time = 1,max_setup_time = 100):
    
    P = {}
    S = {}
    
    N = range(1, N_cardinality+1)
    M = range(1, M_cardinality+1)
    
    for i in M:
        for j in N:
            P[i,j] = r.randint(min_execution_time, max_execution_time)
            
    for i in M:
        for j in N:
            for k in N:
                S[i,j,k] = r.randint(min_setup_time, max_setup_time)
    
    return P,S

N_cardinality = 10
M_cardinality = 2

P_dict, S_dict = generate_instance(N_cardinality=N_cardinality,M_cardinality=M_cardinality,
                                   min_execution_time = 1, max_execution_time = 100,
                                   min_setup_time = 1, max_setup_time = 100)

P_dict, S_dict = generate_instance(N_cardinality=N_cardinality,M_cardinality=M_cardinality)

The sets N,M,N0 are created again to print the results down the line

In [4]:
N = range(1, N_cardinality+1)
M = range(1, M_cardinality+1)
N0 = [i for i in N]
N0.insert(0,0)

# Exact Solution

The problem is solved exactly using the Solver class.

Then, the results are printed. Note that job 0 is a dummy job used to represent the beginning and the end of activities for each machine.

In [5]:
s = Solver(execution_times = P_dict, setup_times = S_dict)
decision_variables,completion_times,maximum_makespan,assignments = s.solve(options=options)
# clear_output(wait=True)
for i in s.N:
    print(f'Job {i} completed at {completion_times[i]}s')

for i in s.M:
    for k in s.N:
        if assignments[i,k] == 1:
            print(f'Job {k} has been assigned to machine {i}')

for i in s.M:
    for j in s.N0:
        for k in s.N0:
            if decision_variables[i,j,k] == 1:
                print(f'Job {j} scheduled in sequence of job {k} in machine {i}')

print(f'The makespan has been minimized to {maximum_makespan}s')

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2519400
Academic license 2519400 - for non-commercial use only - registered to gi___@studenti.units.it
Job 1 completed at 167.0s
Job 2 completed at 70.0s
Job 3 completed at 225.0s
Job 4 completed at 180.0s
Job 5 completed at 221.0s
Job 6 completed at 29.0s
Job 7 completed at 163.0s
Job 8 completed at 102.0s
Job 9 completed at 12.0s
Job 10 completed at 57.0s
Job 5 has been assigned to machine 1
Job 7 has been assigned to machine 1
Job 8 has been assigned to machine 1
Job 9 has been assigned to machine 1
Job 10 has been assigned to machine 1
Job 1 has been assigned to machine 2
Job 2 has been assigned to machine 2
Job 3 has been assigned to machine 2
Job 4 has been assigned to machine 2
Job 6 has been assigned to machine 2
Job 0 scheduled in sequence of job 9 in machine 1
Job 5 scheduled in sequence of job 0 in machine 1
Job 7 scheduled in sequence of job 5 in machine 1
Job 8 scheduled in sequence of job 7

# Mathematical Programming Algorithm (MPA)

The paper cites a decomposition algorithm (Branch-and-check) based on the previous formulation.

It is an exact formulation based on a master algorithm and a sequence algorithm:
- The master algorithm is responsible of assigning jobs to machines
- The sequence problem is responsible of finding the optimal sequence of jobs for each machine

In [6]:
s = MPA(execution_times = P_dict, setup_times = S_dict, t_max=10800) #  max_time = 1800
decision_variables,maximum_makespan = s.solve(options=options)
# clear_output(wait=True)
for i in s.M:
    for j in s.N0:
        for k in s.N0:
            if decision_variables[i,j,k] == 1:
                print(f'Job {j} scheduled in sequence of job {k} in machine {i}')

print(f'The makespan has been minimized to {maximum_makespan}s')

{(1, 1, 1): 85, (1, 1, 2): 72, (1, 1, 3): 91, (1, 1, 4): 6, (1, 1, 5): 74, (1, 1, 6): 53, (1, 1, 7): 60, (1, 1, 8): 35, (1, 1, 9): 61, (1, 1, 10): 42, (1, 2, 1): 10, (1, 2, 2): 47, (1, 2, 3): 8, (1, 2, 4): 88, (1, 2, 5): 83, (1, 2, 6): 83, (1, 2, 7): 70, (1, 2, 8): 13, (1, 2, 9): 9, (1, 2, 10): 55, (1, 3, 1): 15, (1, 3, 2): 7, (1, 3, 3): 83, (1, 3, 4): 22, (1, 3, 5): 9, (1, 3, 6): 1, (1, 3, 7): 87, (1, 3, 8): 40, (1, 3, 9): 6, (1, 3, 10): 18, (1, 4, 1): 49, (1, 4, 2): 76, (1, 4, 3): 20, (1, 4, 4): 1, (1, 4, 5): 58, (1, 4, 6): 60, (1, 4, 7): 64, (1, 4, 8): 92, (1, 4, 9): 68, (1, 4, 10): 3, (1, 5, 1): 38, (1, 5, 2): 11, (1, 5, 3): 35, (1, 5, 4): 32, (1, 5, 5): 37, (1, 5, 6): 88, (1, 5, 7): 76, (1, 5, 8): 83, (1, 5, 9): 66, (1, 5, 10): 1, (1, 6, 1): 71, (1, 6, 2): 31, (1, 6, 3): 93, (1, 6, 4): 1, (1, 6, 5): 46, (1, 6, 6): 15, (1, 6, 7): 16, (1, 6, 8): 51, (1, 6, 9): 49, (1, 6, 10): 58, (1, 7, 1): 38, (1, 7, 2): 45, (1, 7, 3): 93, (1, 7, 4): 43, (1, 7, 5): 4, (1, 7, 6): 33, (1, 7, 7): 18, 

# Fix-and-Optimize approach

The heuristic needs an initial solution. Its quality is proven to be very important in the heuristic's performance. Hence, in the following a greedy solution is found.

In [7]:
greedy = Greedy(execution_times=P_dict, setup_times=S_dict)
solution = greedy.solve()

for i in s.M:
    for j in s.N0:
        for k in s.N0:
            if solution[i,j,k] == 1:
                print(f'Job {j} scheduled in sequence of job {k} in machine {i}')

Job 0 scheduled in sequence of job 8 in machine 1
Job 3 scheduled in sequence of job 6 in machine 1
Job 5 scheduled in sequence of job 0 in machine 1
Job 6 scheduled in sequence of job 7 in machine 1
Job 7 scheduled in sequence of job 5 in machine 1
Job 8 scheduled in sequence of job 9 in machine 1
Job 9 scheduled in sequence of job 3 in machine 1
Job 0 scheduled in sequence of job 2 in machine 2
Job 1 scheduled in sequence of job 10 in machine 2
Job 2 scheduled in sequence of job 1 in machine 2
Job 4 scheduled in sequence of job 0 in machine 2
Job 10 scheduled in sequence of job 4 in machine 2


Now the actual heuristcic can be run with the greedy solution as the starting configuration

In [8]:
m = FixOpt(initial_solution=solution, setup_times=S_dict,execution_times=P_dict, N=N, M=M, N0=N0, 
           subproblem_size_adjust_rate=0.1, t_max = 10, subproblem_runtime_limit=10, subproblem_size=5)
           # WLS_license = True) # In case you are using a WLS Gurobi license

solution, makespan = m.solve()
clear_output(wait=True)
for i in M:
    for j in N0:
        for k in N0:
            if solution[i,j,k] == 1:
                print(f'Job {j} scheduled after job {k} in machine {i}')
print(f'The makespan has been minimized to {makespan}s')

Job 0 scheduled after job 7 in machine 1
Job 5 scheduled after job 10 in machine 1
Job 7 scheduled after job 5 in machine 1
Job 8 scheduled after job 9 in machine 1
Job 9 scheduled after job 0 in machine 1
Job 10 scheduled after job 8 in machine 1
Job 0 scheduled after job 6 in machine 2
Job 1 scheduled after job 4 in machine 2
Job 2 scheduled after job 1 in machine 2
Job 3 scheduled after job 0 in machine 2
Job 4 scheduled after job 3 in machine 2
Job 6 scheduled after job 2 in machine 2
The makespan has been minimized to 225s
